Creating a op property which gives from what operation the current value object is being created.

In [3822]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [3823]:
class Value:
    def __init__(self,data,_children=(),_op = ''):
        self.data = data
        self._prev = _children
        self._op = _op
        self.grad = 0.0
        self._backpropUnit = lambda: None
    
    def __repr__(self):
        return f"Value(data= {self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out =  Value(self.data+other.data,(self,other),'+')
        def _backpropUnit(): 
            self.grad+= 1.0*out.grad
            other.grad+= 1.0*out.grad
        out._backpropUnit = _backpropUnit
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out =  Value(self.data*other.data,(self,other),'*')
        def _backpropUnit(): 
            self.grad +=other.data*out.grad
            other.grad+= self.data*out.grad
        out._backpropUnit = _backpropUnit
        return out

    def __truediv__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        return self*other**-1

    def __rmul__(self,other): #they are in reverse order
        return self*other
    
    def __radd__(self,other):
        return self+other
    
    def __neg__(self):
        return self*-1 

    def __sub__(self,other):
        return self+(-other)
    
    def __rsub__(self,other):
        return -self+other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),_op = 'tanh')
        def _backpropUnit(): 
            self.grad += out.grad*(1-t**2)
        out._backpropUnit = _backpropUnit
        return out
    
    def exp(self):
        x = math.exp(self.data)
        out = Value(x,(self,),_op=f'exp({self.data})')
    
        def _backpropUnit():
            self.grad+= out.grad*out.data
        out._backpropUnit = _backpropUnit
    
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float)) , "only supporting int and float as power"
        out = Value(self.data**other,(self,),f'**{other}')
        def _backpropUnit():
            self.grad+=other * (self.data ** (other - 1)) * out.grad
        out._backpropUnit = _backpropUnit
        return out

    def topologicalOrder(self):
        topo = []
        visited = set()
    
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        return topo

    def printForwardPass(self):
        topo = self.topologicalOrder()
        for v in topo:
            print(f"{v.data} : {v.grad} ")

    def backward(self):
        topo = self.topologicalOrder()
        self.clean()
        self.grad = 1.0
        for node in reversed(topo):
            node._backpropUnit()

    def clean(self):
        nodes = set()
        def _clean(v):
            if v not in nodes:
                nodes.add(v) # ensures atmax only one level deep the repititions will be.
                v.grad = 0
                for child in v._prev:
                    _clean(child)
        _clean(self)

    


In [3824]:


class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        act = sum((xi*wi for xi,wi in zip(x,self.w)),self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return [self.b]+self.w

class Layer:
    def __init__(self,nin,nop):
        self.neurons = [Neuron(nin) for _ in range(nop)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs if len(outs)>1 else outs[0]

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self,nin,nops):
        sz = [nin]+nops
        self.layers =[Layer(sz[i],sz[i+1]) for i in range(len(nops))]
    
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x) # x is updated as the output of the layer(x) which is then fed to another layer.
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]




In [3825]:
n = MLP(3,[4,4,1])
#Running on a simple dataset. 
xs = [[3,5.7,3.2],
      [2.7,4.5,5],
      [9.6,8,7],
      [3,5,7]  
      ]
ys = [1.0,-1.0,0.99,-0.8]
ypred = [n(x) for x in xs]
ypred

[Value(data= 0.8925721165383114),
 Value(data= 0.8940743967741914),
 Value(data= 0.9161426019915461),
 Value(data= 0.8921211752988715)]

In [4097]:
loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))
loss

Value(data= 0.021375970836746677)

In [ ]:
# looping gradient descent of the loss function..
stepsize = 0.001
for k in range(10):
    #forward pass
    ypred = [n(x) for x in xs]
    # loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))/Value(len(ys))
    loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))
    
    #backward pass
    loss.backward()
    
    #update
    mlp_params = n.parameters()
    for p in mlp_params:
        p.data+=-1*stepsize*p.grad
    
    # print(k,loss)



In [4096]:
for ygt,yout in zip(ys,ypred):
    print(f"Expected: {ygt} Predicted: {yout}")

Expected: 1.0 Predicted: Value(data= 0.9739090689530646)
Expected: -1.0 Predicted: Value(data= -0.8989272978265502)
Expected: 0.99 Predicted: Value(data= 0.982163587686943)
Expected: -0.8 Predicted: Value(data= -0.9020692591883944)
